In [1]:
import pandas as pd
import fcStop
import glob
import numpy as np
import os
from functools import reduce
import json
import fcBasicVer
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
years = list(map(str,range(1999,2018)))
with open('../../data/comtrade/countries.json') as f:
    countries = json.load(f)['results']
countries_id = [c['id'] for c in countries]
countries_name = {c['id']:c['text'] for c in countries}

In [3]:
files = {y: glob.glob('../../data/comtrade/Export_data_yearly/AG2/{}/Export_{}_*_0.csv'.format(y,y)) for y in years}

In [4]:
countries_files = {y : [f.split('_')[-2] for f in files[y]] for y in files}

In [5]:
persisting_countries = sorted(reduce(lambda x,y: set.intersection(set(x),set(y)),countries_files.values()))

In [6]:
print len(persisting_countries)

99


In [7]:
countries = [c for c in countries if c['id'] in persisting_countries]

countries_json = {}
countries_json['results'] = countries

with open('../../data/comtrade/pers_countries.json','w') as f:
    json.dump(countries_json,f)

In [8]:
files = {y: [f for f in files[y] if f.split('_')[-2] in persisting_countries] for y in files}

array([[1, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 1, ..., 1, 0, 1],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [32]:
bip_mat

array([[1.80597607e+00, 8.75476260e-01, 2.33482289e-01, ...,
        1.29414574e+01, 7.55937888e-01, 5.84794028e+01],
       [2.58460936e-04, 0.00000000e+00, 3.28208867e-03, ...,
        3.14916606e+02, 4.49648878e+02, 0.00000000e+00],
       [1.30961140e-01, 3.30916980e+00, 7.25365106e-01, ...,
        9.35178853e+00, 5.92602382e-02, 5.86435959e+01],
       ...,
       [6.39145259e-01, 0.00000000e+00, 2.07494962e+01, ...,
        9.84148505e+01, 6.23981473e-02, 3.65120009e+01],
       [8.22209029e-01, 9.10922498e-01, 9.87391689e-01, ...,
        1.13737739e+00, 1.14541917e+00, 1.18591223e+00],
       [1.00001137e+00, 1.00001241e+00, 1.00001148e+00, ...,
        1.00796765e+00, 1.02656438e+00, 0.00000000e+00]])

In [39]:
comtrade_df = {y: pd.concat([pd.read_csv(f) for f in files[y]],ignore_index=1) for y in years}

comtrade_bip = {}
prods = {}
dicts_prod = {}
for y in years:
    comtrade_df[y]['Commodity Code'] = (comtrade_df[y]['Commodity Code']).astype(int)
    products = sorted(set(comtrade_df[y]['Commodity Code']))
    dict_countries = dict(zip(persisting_countries,range(len(persisting_countries))))
    dict_products = dict(zip(products,range(len(products))))
    dicts_prod[y] = dict_products
    prods[y] = products
    bip_mat = np.zeros((len(persisting_countries),len(products)))
    for c,p,v in np.array(comtrade_df[y][['Reporter Code','Commodity Code','Trade Value (US$)']]):
        bip_mat[dict_countries[str(c)],dict_products[p]] += v
    for i in xrange(len(bip_mat[0])): bip_mat[:,i] = bip_mat[:,i]/(bip_mat[:,i].sum())
    
    #for i in xrange(len(bip_mat)):
    #    for j in xrange(len(bip_mat[0])):
    #        bip_mat[i,j] = (bip_mat[i,j]/(bip_mat[i]).sum())/((bip_mat[:,j].sum())/(bip_mat.sum()))
        
    #bip_mat = (bip_mat>1).astype(int)
    
    comtrade_bip[y] = bip_mat

In [40]:
#products = sorted(set(pd.concat(comtrade_df.values(),ignore_index=1)['Commodity Code']))

fitness = pd.DataFrame()
fitness['countries'] = persisting_countries

complexity = pd.DataFrame()
complexity['product'] = prods['1999']

for y in sorted(comtrade_bip):
    fitComplex = fcStop.fitnessComplexityStop(comtrade_bip[y],iteraz=300)
    fitness[y] = [fitComplex[0][dict_countries[c]] for c in persisting_countries]
    complexity[y] = [fitComplex[1][dicts_prod[y][c]] for c in prods['1999']]

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [43]:
fitness.to_csv('YearlyFitnessAG2W.csv',index=None)
complexity.to_csv('YearlyComplexityAG2W.csv',index=None)

In [31]:
#fcBasicVer.computeFQ1(comtrade_bip['1999'],300)[0].shape

In [13]:
fcStop.fitnessComplexityStop(comtrade_bip['1999'],iteraz=300)[0]

0

(99,)

In [16]:
fitness

{'100',
 '108',
 '112',
 '12',
 '124',
 '140',
 '144',
 '152',
 '156',
 '170',
 '188',
 '191',
 '196',
 '203',
 '204',
 '208',
 '218',
 '222',
 '233',
 '246',
 '251',
 '268',
 '270',
 '276',
 '300',
 '31',
 '32',
 '320',
 '328',
 '344',
 '348',
 '352',
 '36',
 '360',
 '372',
 '376',
 '381',
 '384',
 '388',
 '392',
 '398',
 '40',
 '400',
 '410',
 '422',
 '428',
 '440',
 '442',
 '450',
 '454',
 '458',
 '462',
 '470',
 '480',
 '484',
 '490',
 '498',
 '504',
 '508',
 '51',
 '512',
 '52',
 '528',
 '554',
 '558',
 '56',
 '579',
 '600',
 '604',
 '608',
 '616',
 '620',
 '642',
 '643',
 '659',
 '662',
 '670',
 '678',
 '68',
 '686',
 '699',
 '702',
 '703',
 '705',
 '724',
 '752',
 '757',
 '76',
 '788',
 '792',
 '8',
 '800',
 '804',
 '818',
 '826',
 '834',
 '84',
 '842',
 '858'}

In [33]:
comtrade_bip['1999'].shape

(99, 174)